In [ ]:
from Read_NZ_Stats import Read_NZ_Stats

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib

In [ ]:
circuit = 195
filename = "FeynmanDet/build/apps/NZ_paths_" + str(circuit) + ".data"

header, paths, pathsW = Read_NZ_Stats (filename)

print("Total NZ paths read:", len(paths))

print("Header:")
print("L =", header["L"])
print("NQ =", header["NQ"])
print("< ", header["state_f"], "| U |", header["state_i"], " > = ", header["aR"], " + i ", header["aI"])
print("X_magic =", header["X_magic"])
print("total_paths =", header["total_paths"])
print("total_NZ_paths =", header["total_NZ_paths"])
print()

In [ ]:
# convert from read format to display format

# paths is OK

# compute weight magnitude from pathsW
pW_m = []
pW_Re = []
pW_Im = []
pW_ph = []
for a in pathsW:
    Re = a[0]
    pW_Re.append(Re)
    Im = a[1]
    pW_Im.append(Im)
    W_m = np.sqrt(Re**2 + Im**2)
    pW_m.append(W_m)
    W_ph = np.arctan2(Im,Re)
    pW_ph.append(W_ph)

In [ ]:

def inverse_participation_ratio(weights, normalize=True):
    """
    Compute the inverse participation ratio (IPR).

    Parameters
    ----------
    weights : array-like
        Real or complex weights.
    normalize : bool
        If True, normalize by L1 norm before computing IPR.
        If False, assumes weights already normalized.

    Returns
    -------
    float
        Inverse participation ratio.
    """
    w = np.asarray(weights)
    
    if normalize:
        norm = np.sum(w)
        if norm == 0:
            return 0.0
        w = w / norm
        return np.sum(w**2)
    else:
        return np.sum(w**2)

def effective_number_of_paths(weights):
    ipr = inverse_participation_ratio(weights, normalize=True)
    return 1.0 / ipr if ipr > 0 else 0.0

In [ ]:


def plot_paths(paths, weights, L, N, Sparsity, cmap_name="viridis"):
    """
    paths   : list of lists (each of length L+1)
    weights : list or array of positive weights (same length as paths)
    L       : path length
    N       : number of states
    """

    fig, ax = plt.subplots(figsize=(8, 5))

    weights = np.array(weights)
    norm = mcolors.Normalize(vmin=weights.min(), vmax=weights.max())
    #cmap = cm.get_cmap(cmap_name)
    cmap = matplotlib.colormaps[cmap_name]
    #sm = cm.ScalarMappable(norm=norm, cmap=cmap)

    # --- Draw nodes ---
    for l in range(L + 1):
        x = np.full(N, l)
        y = np.arange(N)
        ax.scatter(x, y, s=0.01, color="black", zorder=3)
        
    for S, W in zip(paths, weights):
        l_vals = np.arange(-1, L + 2)
        #color = cmap(norm(W))
        #ax.plot(l_vals, S, '-', color=color, linewidth=0.1)
        S_l = list([header["state_i"]]) + list(S) + list([header["state_f"]])
        ax.plot(l_vals, S_l, '-', color='b', linewidth=0.1, alpha=0.8, zorder=2)

    # Discrete axis formatting
    ax.set_xlim(0, L)
    ax.set_ylim(-0.5, N - 0.5)
    ax.set_xticks(range(-1, L + 2))
    ax.set_yticks([0]+[2**i for i in range(header["NQ"]+1)])
    ax.set_xlabel("Layer")
    ax.set_ylabel("State")
    ax.set_title("Circuit " + str(circuit) + ": Non Zero Paths ("+ str(header["total_NZ_paths"])+ ")")
    # Sparsity metric
    ax.text(0.85, 0.97,
                f"S = {Sparsity:.1e} %",
                transform=ax.transAxes,
                verticalalignment='top',
                bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 2},
               fontsize=10)

    # Add colorbar
    #cbar = fig.colorbar(sm, ax=ax)
    #cbar.set_label("Path weight W")

    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Example parameters
#L = 6
N = 2**header["NQ"]

# Random example paths (P small)
#P = 12
#paths = [np.random.randint(0, N, size=L+1) for _ in range(P)]
#weights = np.random.rand(P)

N_eff = effective_number_of_paths(pW_m)
print ("Neff = ", N_eff)
Sparsity = header["total_NZ_paths"] * 100./ header["total_paths"]

plot_paths(paths, pW_m, header["L"]-1, N, Sparsity, cmap_name="plasma")

In [ ]:
import matplotlib.cm as cm

def plot_complex_weights_polar(pw_m, pw_ph, aR, aI, X_magic,
                               scale_linewidth=False,
                               log_radius=False):
    """
    weights : array-like of complex numbers (non-normalized)
    scale_linewidth : scale thickness with magnitude
    log_radius : use log(|W|) for radial coordinate (useful for wide dynamic range)
    """

    magnitudes = np.asarray(pw_m)
    phases = np.asarray(pw_ph)

    if log_radius:
        magnitudes = np.log10(magnitudes + 1e-16)  # avoid log(0)

    # Normalize phase for cyclic colormap
    norm = mcolors.Normalize(vmin=-np.pi, vmax=np.pi)
    cmap = cm.get_cmap("twilight")  # cyclic map ideal for phase

    fig = plt.figure(figsize=(7, 7))
    ax = fig.add_subplot(111, projection='polar')

    for r, theta in zip(magnitudes, phases):
        color = cmap(norm(theta))
        lw = 1.3
        if scale_linewidth:
            lw = 0.5 + 3.0 * (r / magnitudes.max())

        ax.plot([theta, theta], [0, r],
                color=color, marker='o',
                linewidth=lw,
                alpha=1)
    # add final amplitude
    theta = np.arctan2(aI, aR)
    r = magnitudes.max()
    ax.plot([theta, theta], [0, r+0.1*r],
            color='black',
            linestyle='dotted',
            linewidth=4,
            label="Amplitude")
    #ax.scatter([theta], [r],
    #           color="black",
    #           s=100,
    #           zorder=5)

    # X_Magic metric
    ax.text(0.0, 1.,
                f"X_magic = {X_magic:.1f}",
                transform=ax.transAxes,
                verticalalignment='top',
                bbox={'facecolor': 'white', 'alpha': 0.5, 'pad': 3},
               fontsize=11)

    ax.set_title("Circuit " + str(circuit) + ": Complex Path Weights", pad=10)
    ax.legend(loc='upper right')

    # Phase colorbar
    sm = cm.ScalarMappable(norm=norm, cmap=cmap)
    cbar = fig.colorbar(sm, ax=ax, pad=0.1)
    cbar.set_label("Phase (radians)")

    plt.tight_layout()
    plt.show()

In [ ]:

plot_complex_weights_polar (pW_m, pW_ph, header["aR"], header["aI"], round(header["X_magic"],1), log_radius=False)